# Optimize the model Size and Performance for mobile platforms


The objective of this notebook is to convert the pytorch model to format compatible to mobile platforms.
1. convert the PyTorch model to ONNX format
   
   ONNX (Open Neural Network Exchange) is open format for exchange of Deep Learning model between different Frameworks. In other words it is format which is widely used for exchanging one model from one framework to other framework. We will first convert model from Pytorch to ONNX so we can transfer it to another framework.
The Process of converting to ONNX is that we make a random dummy input and pass it through the model in pytorch and then pass it throught the function of onnx export to export it to a .onnx file. The shape of the dummy input should be (1, shape of single images) in detail the input would be (1, number of color channels, width of picture , height of picture )`

In [35]:
# helper functions for this notebook
def gvd(variable):
    print("The data type of this variable is " + str(type(variable)))
import time
def mills():
    return int(round(time.time() * 1000))

In [16]:
import torch
import torch.onnx
import torchvision.models as models
import time
import onnx

In [3]:

# Load the trained PyTorch model
model = models.resnet152(pretrained=False)
num_ftrs = model.fc.in_features
out_ftrs = 2  # Binary classification
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, out_ftrs),
    torch.nn.LogSoftmax(dim=1)
)
model.load_state_dict(torch.load('final_model.pth'))
model.eval()

# Example input with batch size of 1
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format with optimizations
start_time = time.time()
torch.onnx.export(
    model, 
    dummy_input, 
    "model.onnx", 
    verbose=False, 
    export_params=True, 
    do_constant_folding=True
)
end_time = time.time()

print(f"Export time: {end_time - start_time} seconds")


/home/thembo/college/cataract/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/thembo/college/cataract/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Export time: 3.272263526916504 seconds



## Run ONNX model with ONNX runtime

In [36]:
import onnxruntime as nxrun

sess = nxrun.InferenceSession('./model.onnx')
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
start_time = mills()
# run onnx model with onnx runtime python
result = sess.run(None, {input_name: dummy_input.numpy()})

end_time = mills()
print("model single inference in milliSeconds on onnxruntime", end_time - start_time)
print("Output", result)

model single inference in milliSeconds on onnxruntime 88
Output [array([[-0.5618211, -0.844365 ]], dtype=float32)]


## Convert from ONNX to TensorFlow FreezeGraph


We will use onnx-tf to convert model

onnx_tf is the library build by onnx team which is used to transfer the model from onnx to tensorflow it can create a backend to enable the model to run with tensorflow. We will first load the saved .onnx model file with onnx.load and the by using prepare function of onnx_tf prepare that loaded model to be run by tensorflow. and by using the export_graph function of that prepared backend we can export this model in a file format with .tf extension supported by Original Tensorflow


In [38]:
# install onnx and onnx-tf
!pip install --upgrade onnx | tail -n 2
!pip install  onnx-tf==1.5.0 | tail -n 2

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [52]:
# see versions of all tensorflow libraries installed
!pip list | grep 'flow'

tensorflow                    2.5.0
tensorflow-addons             0.23.0
tensorflow-cpu                2.16.1
tensorflow-datasets           4.9.3
tensorflow-estimator          2.5.0
tensorflow-hub                0.16.1
tensorflow-io-gcs-filesystem  0.37.0
tensorflow-metadata           1.15.0
tensorflow-model-optimization 0.8.0
tensorflow-probability        0.24.0
tensorflow-text               2.15.0


In [53]:
import onnx
import tensorflow as tf

In [54]:
!onnx-tf convert -i "./model.onnx" -o  './final_model.pb' 

2024-05-30 10:54:57.200026: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-30 10:54:57.200051: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/thembo/college/cataract/lib/python3.9/site-packages/tensorflow/python/keras/engine/training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.19.5)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top
/home/thembo/college/cataract/lib/python3.9/site-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)
Traceback (most recent call last):
  File "/home/

In [56]:
# load the model saved in onnx format
from onnx_tf.backend import prepare

model_onnx = onnx.load('model.onnx')
onnx.checker.check_model(model_onnx)

# prepare model for exporting to tensorFlow using tensorFlow backend
tf_rep = prepare(model_onnx)
start_time = mills()
print(tf_rep.run(dummy_input))
end_time = mills()
print(end_time - start_time)
print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model

# # export tensorFlow backend to tensorflow tf file
tf_rep.export_graph('final_model.pb')
tf_rep.run(dummy_input)

AttributeError: module 'tensorflow' has no attribute 'contrib'